In [ ]:
!pip install -q gradio
!pip install -q accelerate peft
!pip install -q -U bitsandbytes
!git clone https://github.com/parsakzr/codellm.git
!pip install git+https://github.com/huggingface/transformers

In [ ]:
# import locale
# def getpreferredencoding(do_setlocale = True):
#     return "UTF-8"
# locale.getpreferredencoding = getpreferredencoding

In [ ]:
import transformers

transformers.is_bitsandbytes_available()

True

Custom code to handle Model inputs/outputs

In [ ]:
def generate_prompt(text: str, model_name, prompt_template: str = ""):
    if prompt_template:
        try:
            prompt = prompt_template.format(text)
        except KeyError:
            # if the prompt template is not valid, use the original prompt
            print(
                "Invalid prompt template, using original prompt. Make sure to include {} in the template."
            )
            return text
        return prompt

    if "codegen" in model_name.lower() or "phi" in model_name.lower():
        # style "alpaca":
        system_msg = f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n"
        return system_msg + f"### Instruction: Write a high quality Python code to solve the following problem:\n{text}\n\n### Response:\n"

    if "mistral" in model_name.lower():
        system_msg = "Below is an instruction that describes a programming task. Write a response code that appropriately completes the request.\n"
        return f"<s>[INST] {system_msg}\nWrite a high quality Python code to solve the following problem:\n{text} [/INST]"

    return text


def filter_code(completion: str, prompt: str = None, template: str = "") -> str:
    try:
        code = completion
        if prompt is not None:
            # remove the prompt, since it's a completion model
            code = code.replace(prompt, "")

        if template == "alpaca":
            ## Remove boilerplate for the function, reused pure_mode in generation_pipeline
            # select the text between the two '''
            code = code.split("'''")[1]
            # remove the first line (which is the language)
            code = "\n".join(code.split("\n")[1:])

        if template == "phi":
            ## Remove boilerplate for the function, reused pure_mode in generation_pipeline
            # select the text between the two '''
            code = code.split("```")[1]
            # remove the first line (which is the language)
            code = "\n".join(code.split("\n")[1:])

        if template == "mistral":
            # get the code inside [CODE] ... [/CODE]
            code = code.split("[CODE]")[1]
            code = code.split("[/CODE]")[0]


        ## The program tends to overwrite, we only take the first function
        code = code.lstrip("\n")
        return code.split("\n\n")[0]
    except Exception as e:
        print(e)
        return code
    finally:
        return code


In [ ]:
"""
Getting all the models (4bit quantized)
"""
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
import torch
from codellm.model import EvalModel

model_ids = {
    "codegen": "parsak/codegen-350M-mono-lora-instruction",
    "mistral": "parsak/mistral-code-7b-instruct",
    "phi" : "parsak/phi-2-code-instruct"
}

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

mistralModel = AutoModelForCausalLM.from_pretrained(
    model_ids["mistral"], quantization_config=bnb_config, device_map={"": 0}
)
# tokenizer
mistralTokenizer = AutoTokenizer.from_pretrained(model_ids["mistral"])
mistralTokenizer.pad_token = mistralTokenizer.eos_token
mistralTokenizer.padding_side = "right"

mistralEvalModel = EvalModel()
mistralEvalModel.load_from(mistralModel, mistralTokenizer)
mistralEvalModel.model_name = model_ids['mistral'].split('/')[-1]

# CodeGen
codegenModel = AutoModelForCausalLM.from_pretrained(
    model_ids["codegen"], quantization_config=bnb_config, device_map={"": 0}
)
# tokenizer
codegenTokenizer = AutoTokenizer.from_pretrained(model_ids["codegen"])
codegenTokenizer.pad_token = codegenTokenizer.eos_token
codegenTokenizer.padding_side = "right"

codegenEvalModel = EvalModel()
codegenEvalModel.load_from(codegenModel, codegenTokenizer)
codegenEvalModel.model_name = model_ids['codegen'].split('/')[-1]


phiModel = AutoModelForCausalLM.from_pretrained(
    model_ids["phi"], quantization_config=bnb_config, device_map={"": 0}, trust_remote_code=True)

# tokenizer
phiTokenizer = AutoTokenizer.from_pretrained(model_ids["phi"])
phiTokenizer.pad_token = phiTokenizer.eos_token
phiTokenizer.padding_side = "right"

phiEvalModel = EvalModel()
phiEvalModel.load_from(phiModel, phiTokenizer)
phiEvalModel.model_name = model_ids['phi'].split('/')[-1]

In [ ]:
print(mistralEvalModel.model_name)
print(codegenEvalModel.model_name)
print(phiEvalModel.model_name)

mistral-code-7b-instruct
codegen-350M-mono-lora-instruction
phi-2-code-instruct


In [ ]:
mistralEvalModel.generate(prompt='Write a function that calculates the standard deviation of a set', verbose=True, pure_mode=False)

In [ ]:
codegenEvalModel.generate(prompt='Write a function that prints the first n elements of Fibonacci series', verbose=True, pure_mode=False)

In [ ]:
phiEvalModel.generate(prompt='Write a function that prints the first n elements of Fibonacci series', verbose=True, pure_mode=False)

In [ ]:
print(filter_code(mistralEvalModel.generate("Write a function that calculates the area of the circle"), template="mistral"))

In [ ]:
import gradio as gr

model = {
    mistralEvalModel.model_name: mistralEvalModel,
    codegenEvalModel.model_name: codegenEvalModel,
    phiEvalModel.model_name: phiEvalModel
}
model_template = {
    codegenEvalModel.model_name: "alpaca",
    mistralEvalModel.model_name: "mistral",
    phiEvalModel.model_name: "phi"
}
choices = list(model.keys())
dropdown = gr.Dropdown(
    label="Models", choices=choices, value=phiEvalModel.model_name
)

def respond(message, chat_history, additional_inputs):
    if message == "" or None:
        return "Please enter a prompt"


    model_name = additional_inputs
    modelEval = model[model_name]
    template = model_template[model_name]

    prompt = generate_prompt(message, modelEval.model_name,)
    response = modelEval.run(message)
    response = filter_code(response, template=template)
    return f"Here's an example code:\n\n```python\n{response}\n```"


interface = gr.ChatInterface(
    respond,
    retry_btn=gr.Button(value="Retry"),
    undo_btn=None,
    clear_btn=gr.Button(value="Clear"),
    additional_inputs=[dropdown],
)


In [ ]:
respond("verilen bir listenin ortalamasını bulan fonksiyonu yaz","", mistralEvalModel.model_name)

In [ ]:
# run the interface
interface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://8cce120660f4599936.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
interface.close()

Closing server running on port: 7860
